# Importe de librerias

In [ ]:
import lightning as L
from pytorch_lightning.loggers import CSVLogger

from sklearn.model_selection import train_test_split
import torch
import optuna

torch.set_float32_matmul_precision('high')

/home/exodia/miniconda3/envs/ML-TF-G/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import LitPriceData
from trainer import LitTrainer

In [ ]:
from utils import splitData

In [ ]:
name = "bitcoin"

def objective(trial):   
    hidden_size = trial.suggest_int("hidden_size", 32, 128)
    num_layers = trial.suggest_int("num_layers", 2, 5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout= trial.suggest_float("dropout", 0.0, 0.5)
    
    sequence_length = trial.suggest_int("sequence_legth", 12, 72, step=12)

    logger = CSVLogger("lightning_logs", name="optuna")
    
    model = LitTrainer(hidden_size=hidden_size, num_layers=num_layers, lr=lr, dropout=dropout)
    X, y = splitData(name, sequence_length=sequence_length)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    dataModule = LitPriceData(X_train, y_train, X_val, y_val, num_workers=4)
       
    trainer = L.Trainer(max_epochs=50, accelerator="auto", logger=logger, enable_progress_bar=False, enable_model_summary=False, enable_checkpointing=False, log_every_n_steps=2)
    trainer.fit(model=model, datamodule=dataModule)
    
    return trainer.callback_metrics["val_loss"].item()

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

print("Best trial:")
print(study.best_params)

In [12]:
best_params = study.best_params
best_params

{'hidden_size': 110,
 'num_layers': 2,
 'lr': 0.006828760040299548,
 'dropout': 0.15845603432711983,
 'sequence_legth': 24}

In [11]:
optuna.visualization.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
                    111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
                    123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
                    135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
                    147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
                    159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
                    171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
                    183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
                    195, 196, 197, 198, 199],
              'y': [0.015474977903068066, 0.018035802990198135,
                    0.01172280590981245, 0.015552688390016556,
                    0.012422272004187107, 0.01193363219499588,
                    0.012360180728137493, 0.012184040620923042,
                    0.012760867364704609, 0.021831316873431206,
                    0.011060379445552826, 0.012907866388559341,
                    0.013342889957129955, 0.010852104984223843,
                    0.013775920495390892, 0.01114918477833271,
                    0.010638038627803326, 0.010312814265489578,
                    0.011704572476446629, 0.012167590670287609,
                    0.009568741545081139, 0.01032345276325941,
                    0.010330011136829853, 0.009103646501898766,
                    0.01277197152376175, 0.008306256495416164,
                    0.008161399513483047, 0.010389626957476139,
                    0.011550645343959332, 0.01066564954817295, 0.01174628734588623,
                    0.009237675927579403, 0.013045659288764, 0.00931067205965519,
                    0.009164204820990562, 0.010224045254290104,
                    0.013479546643793583, 0.01550319604575634,
                    0.012935811653733253, 0.011722207069396973,
                    0.009785306639969349, 0.008815539069473743,
                    0.008268902078270912, 0.009826598688960075,
                    0.008463452570140362, 0.009920714423060417,
                    0.012363084591925144, 0.00998581200838089,
                    0.010012906976044178, 0.016361532732844353,
                    0.008652585558593273, 0.008778784424066544,
                    0.011016516014933586, 0.010955131612718105,
                    0.00995897501707077, 0.008902725763618946, 0.01302336435765028,
                    0.010360246524214745, 0.009426789358258247,
                    0.02180483378469944, 0.00988241471350193, 0.008514647372066975,
                    0.008843361400067806, 0.008444142527878284,
                    0.007936238311231136, 0.009695986285805702,
                    0.009727188386023045, 0.008505918085575104,
                    0.02034570276737213, 0.010286790318787098,
                    0.009582074359059334, 0.008004137314856052,
                    0.013821275904774666, 0.009703856892883778,
                    0.009877162054181099, 0.009186320006847382,
                    0.012237993068993092, 0.009345858357846737,
                    0.011266681365668774, 0.007100023794919252,
                    0.010422640480101109, 0.008978435769677162,
                    0.009113254025578499, 0.008881616406142712,
                    0.008451659232378006, 0.012066529132425785,

In [9]:
optuna.visualization.plot_parallel_coordinate(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [0.006468992214649916, 0.03281501680612564],
                              'values': [0.015474977903068066,
                                         0.018035802990198135, 0.01172280590981245,
                                         0.015552688390016556,
                                         0.012422272004187107, 0.01193363219499588,
                                         0.012360180728137493,
                                         0.012184040620923042,
                                         0.012760867364704609,
                                         0.021831316873431206,
                                         0.011060379445552826,
                                         0.012907866388559341,
                                         0.013342889957129955,
                                         0.010852104984223843,
                                         0.013775920495390892, 0.01114918477833271,
                                         0.010638038627803326,
                                         0.010312814265489578,
                                         0.011704572476446629,
                                         0.012167590670287609,
                                         0.009568741545081139, 0.01032345276325941,
                                         0.010330011136829853,
                                         0.009103646501898766, 0.01277197152376175,
                                         0.008306256495416164,
                                         0.008161399513483047,
                                         0.010389626957476139,
                                         0.011550645343959332, 0.01066564954817295,
                                         0.01174628734588623, 0.009237675927579403,
                                         0.013045659288764, 0.00931067205965519,
                                         0.009164204820990562,
                                         0.010224045254290104,
                                         0.013479546643793583, 0.01550319604575634,
                                         0.012935811653733253,
                                         0.011722207069396973,
                                         0.009785306639969349,
                                         0.008815539069473743,
                                         0.008268902078270912,
                                         0.009826598688960075,
                                         0.008463452570140362,
                                         0.009920714423060417,
                                         0.012363084591925144, 0.00998581200838089,
                                         0.010012906976044178,
                                         0.016361532732844353,
                                         0.008652585558593273,
                                         0.008778784424066544,
                                         0.011016516014933586,
                                         0.010955131612718105, 0.00995897501707077,
                                         0.008902725763618946, 0.01302336435765028,
                                         0.010360246524214745,
                                         0.009426789358258247, 0.02180483378469944,
                                         0.00988241471350193, 0.008514647372066975,
                                         0.008843361400067806,
                                         0.008444142527878284,
                                         0.007936238311231136,
                                         0.009695986285805702,
                                         0.009727188386023045,
                                         0.008505918085575104, 0.02034570276737213,
                                         0.010286790318787098,
                                         0.009582074359059334,
         

In [10]:
optuna.visualization.plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [dropout (FloatDistribution):
                                0.11170077559959829<extra></extra>, hidden_size
                                (IntDistribution):
                                0.12306668133045205<extra></extra>, lr
                                (FloatDistribution):
                                0.15042441020319566<extra></extra>, sequence_legth
                                (IntDistribution):
                                0.3070351601417819<extra></extra>, num_layers
                                (IntDistribution):
                                0.30777297272497206<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [0.11, 0.12, 0.15, 0.31, 0.31],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.11170077559959829, 0.12306668133045205, 0.15042441020319566,
                    0.3070351601417819, 0.30777297272497206],
              'y': [dropout, hidden_size, lr, sequence_legth, num_layers]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})